In [ ]:
#Máquina de vetores de suporte

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from sklearn.multioutput import MultiOutputClassifier

# Carrega a tabela de dados com a coluna "Title" e "Genre"
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tcc/Netflix Dataset Latest 2021.csv")

# Tratamento de células vazias
data = data.dropna(subset=['Title', 'Genre'])  # Remove linhas com valores vazios em Title ou Genre

# Divide os dados em recursos (X) e rótulos (y)
X = data['Title']
y = data['Genre']

# Pré-processamento dos títulos
nltk.download('stopwords')
nltk.download('punkt')

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')

def preprocess_text(text):
    # Verifica se é um valor válido
    if pd.notnull(text):
        # Tokenização
        tokens = word_tokenize(text)
        # Remoção de stopwords e stemming
        tokens_processed = [stemmer.stem(token) for token in tokens if token.lower() not in stop_words]
        # Reconstrução do texto processado
        text_processed = ' '.join(tokens_processed)
        return text_processed
    else:
        return ''

X_processed = X.apply(preprocess_text)

# Realiza a vetorização dos títulos usando TF-IDF
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X_processed)

# Realiza a codificação multilabel dos rótulos
mlb = MultiLabelBinarizer()
y_bin = mlb.fit_transform(y.str.split(', '))

# Cria um classificador SVM para classificação multilabel
clf = MultiOutputClassifier(SVC())

# Treina o classificador com todos os dados vetorizados e rótulos binarizados
clf.fit(X_vec, y_bin)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


MultiOutputClassifier(estimator=SVC())

In [ ]:
# Avalia a precisão do modelo nos dados de treinamento
y_pred = clf.predict(X_vec)
accuracy = accuracy_score(y_bin, y_pred)
print("Precisão do Modelo:", accuracy)
#0-1

Precisão do Modelo: 0.34085106382978725


In [ ]:
# Exemplo de previsão para um novo título de filme
novo_titulo = ""
novo_titulo_processed = preprocess_text(novo_titulo)
novo_titulo_vec = vectorizer.transform([novo_titulo_processed])

previsao = clf.predict(novo_titulo_vec)
generos_preditos = mlb.inverse_transform(previsao)
print("Gêneros Preditos:", generos_preditos)

Gêneros Preditos: [('Horror', 'Mystery', 'Thriller')]
